In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20641020
paper_name = 'heuck_kanter_2010' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/yea_1780_sm_supportinginformations1.xlsx', sheet_name='Table 1', skiprows=1)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4863 x 11


In [10]:
original_data.head()

,strain,Cs+ in nmol/1x107 cells,SD (Cs+),Z score (Cs+),Unnamed: 4,Sr2+ in nmol/1x107 cells,SD (Sr2+),Z score (Sr2+),Unnamed: 8,Unnamed: 9,Unnamed: 10
0,BY4741,0.92,0.13,0.00,NaN,1.5,0.22,0.00,NaN,Wild type,NaN
1,YAL002W,0.63,0.03,-2.06,NaN,3.61,0.16,9.36,NaN,NaN,NaN
2,YAL004W,1.01,0.12,0.67,NaN,1.68,0.42,0.81,NaN,NaN,NaN
3,YAL005C,1.05,0.03,0.96,NaN,1.8,0.11,1.32,NaN,NaN,NaN
4,YAL007C,1.15,0.19,1.63,NaN,1.79,0.38,1.27,NaN,NaN,NaN


In [11]:
original_data['orf'] = original_data['strain'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

             strain Cs+ in nmol/1x107 cells  SD (Cs+)  Z score (Cs+)  \
index_input                                                            
0            BY4741                    0.92      0.13            0.0   

            Unnamed: 4 Sr2+ in nmol/1x107 cells  SD (Sr2+)  Z score (Sr2+)  \
index_input                                                                  
0                  NaN                      1.5       0.22             0.0   

             Unnamed: 8 Unnamed: 9  Unnamed: 10     orf  
index_input                                              
0                   NaN  Wild type          NaN  BY4741  


In [15]:
original_data = original_data.loc[t,:]

In [16]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data = original_data[['Z score (Cs+)','Z score (Sr2+)']].copy()

In [19]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(4761, 2)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [16281, 16284]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,16281,16284
data_type,value,value
orf,,
YAL002W,-2.06,9.36
YAL004W,0.67,0.81
YAL005C,0.96,1.32
YAL007C,1.63,1.27
YAL008W,1.98,2.42


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16281,16284
,data_type,value,value
gene_id,orf,,
2,YAL002W,-2.06,9.36
1863,YAL004W,0.67,0.81
4,YAL005C,0.96,1.32
5,YAL007C,1.63,1.27
6,YAL008W,1.98,2.42


# Normalize

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16281,16284,16281,16284
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,-2.06,9.36,-1.379951,6.214203
1863,YAL004W,0.67,0.81,0.598403,0.735138
4,YAL005C,0.96,1.32,0.808558,1.061959
5,YAL007C,1.63,1.27,1.294088,1.029918
6,YAL008W,1.98,2.42,1.547723,1.766868


# Print out

In [31]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [32]:
from IO.save_data_to_db3 import *

In [33]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 20641020...
Inserting the new data...


100%|██████████| 2/2 [00:15<00:00,  7.57s/it]

Updating the data_modified_on field...
